# Import libraries, set options, connect to DB

In [133]:
# Configuration code for datawrangling
import pandas as pd
import os
import numpy as np
from datetime import datetime
from geocode import geocode
from mapToPoly import mapToPoly
pd.set_option('display.max_row', 30000)
import csv

# Configuration code in order to connect to the database
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker
from database_setup import Itenerary, Base

passWord = os.environ['my_password']
DATABASE_URI = 'postgres+psycopg2://maxcarey:' + passWord + '@localhost:5432/totago'
engine = create_engine(DATABASE_URI)

#engine = create_engine('sqlite:///totagoData.db')

# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine

DBSession = sessionmaker(bind=engine)

session = DBSession()

# Read in data as pandas data frame, selecting only certain fields

In [134]:
fields = ['distinct_id', 'numItinerariesReturned', 'departureDate', 'startFromLocation', 'selectedDestination_id', 'selectedDestination_name', 'time']

In [135]:
df = pd.read_csv('generated_itineraries.csv', usecols = fields)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Wrange field: destinationIDs

In [136]:
# Replace all of the NAs for destinationIDs with 0
df.selectedDestination_id.fillna(0, inplace = True)

# Remove the 2 cases where the string says null
# Great tutorial here: https://www.youtube.com/watch?v=2AFGPdNn4FM
df = df[df.selectedDestination_id != 'null']

# Convert destinationIDs column to an integer value
df['selectedDestination_id'] = df.selectedDestination_id.astype(int)



# Wrangle field: numItenerariesReturned

In [137]:
# Replace all of the NAs for numItinerariesReturned with 1
df.numItinerariesReturned.fillna(1, inplace = True)

# Convert from float to integer
df['numItinerariesReturned'] = df.numItinerariesReturned.astype(int)

# Select, only observatiosn where this field is greater than 0 (now that the NAs are gone)


# Wrangle Field: Destination Name

In [138]:
#Replace all of the NAs in
df.selectedDestination_name.fillna("", inplace = True)

print("Number of rows before departure date: ")
print(len(df))


Number of rows before departure date: 
28138


# Wrangle Field: departureDate

In [139]:
#Convert destinationIDs column to an integer value
# It looks like there were some complex rows being held in here before, I thought that when df.dtypes returned object that
# meant string but apprently not
df['departureDate'] = df.departureDate.astype(str)

print("number of rows before removal of anamoulous departureDate cases")
print(len(df))


# IT looks like there are some cases where this field is blank, says nan, is in format 24503, or in format "masked" 
# We need to remove these cases from the data frame
# I can see that some blank rows are still printed out.
df = df[df.departureDate != '']
df = df[df.departureDate != 'nan']
df = df[df.departureDate != '24503']
df = df[df.departureDate != '[masked]']


print("number of rows after removal of anamoulous departureDate cases")
print(len(df))

# Create a function extractDate that extracts the first ten characters of an input string
def extractDate(dateString):
    extractedDate = dateString[0:10]
    if len(extractedDate) < 10:
        print(extractedDate)
    return extractedDate

''' Code to test if the extractDate function works

# Apply this function to create  a new column
df['departureDateFixed'] = df.departureDate.apply(extractDate)

cols = ['distinct_id', 'departureDate', 'departureDateFixed', 'numItinerariesReturned', 'selectedDestination_id', 'selectedDestination_name', 'startFromLocation']

df = df[cols]
'''

# Override departure date extracting all of the null time stamps
df['departureDate'] = df.departureDate.apply(extractDate)


# Convert departure date into a time object in pandas
#See here: https://stackoverflow.com/questions/26763344/convert-pandas-column-to-datetime
# Though this actually might not need to be done
#df['departureDate'] = df.departureDate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))




#df['departureDate'] = datetime.strptime(df['departureDate'], '%Y-%m-%-d')  
#df['departureDate'] = pd.to_datetime(df['departureDate'], format = '%Y-%m-%-d')



number of rows before removal of anamoulous departureDate cases
28138
number of rows after removal of anamoulous departureDate cases
28130


# Wrangle Field: distinctID

In [140]:
#It turns out distinc_id correpsonds to a user

# Therefore, create a coloumn that combines the unix time stamp with distinct_id so that we have a primary key for database
df["primary_key"] = df["distinct_id"] + "-" + df["time"].map(str)


vc = df.primary_key.value_counts()
print(vc[vc > 1])


unique_keys = df.primary_key.unique()
#print(len(unique_keys))

#df.head(n = len(df))

156c0427d8411b-089cb3f5e0f3f-1c114a5c-13c680-156c0427d85c5-1472086519       3
16017f5f592536-094b655d7b8ce4-7636321b-4a640-16017f5f5932bd-1512207331      3
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989343                             3
15ae497624281a-06e6a097b99f59-5e4f2b18-ff000-15ae4976243a16-1489889250      3
68f3348e-32ff-4756-a6b9-fbf722d5bf76-1460184823                             3
2578f3c5-2bd8-4d7e-b05e-d82eb4e24632-1448989351                             2
6059a95c-9075-49c1-bfce-dc5420759ed3-1469199106                             2
acc70a5e-fced-43ec-9225-b5b13b9d3103-1451167936                             2
2b9795d4-912a-4c5b-b628-1ccd366c488f-1466266682                             2
9a942f21-ad25-4129-b760-46bcec9e631d-1448989392                             2
f763ca4a-e65f-4054-8a7a-8682bc627b15-1474711830                             2
f67face6-b99b-4ddc-8326-3dbe6998c01c-1490893501                             2
cb9dcbce-753b-4fe5-a91a-1a58a779263c-1494110966                 

# Create a subset of the datle with sample method to test geocode and database entry logic

In [141]:
#Out put the entire database
#df.head(len(df))

len(df)

28130

In [142]:
#Create a random sample of the database, these entries will be added to the database in the next section
sampleDf = df.sample(500)

# Output this random sample
sampleDf.head(len(sampleDf))    

,departureDate,distinct_id,numItinerariesReturned,selectedDestination_id,selectedDestination_name,startFromLocation,time,primary_key
13993,2017-07-20,15d5be0a3f6e2-05749e0830b429-3065780b-13c680-1...,1,4,West Tiger No. 3,"Seattle, WA, United States",1500461033,15d5be0a3f6e2-05749e0830b429-3065780b-13c680-1...
720,2016-06-05,0dec1da9-0ad4-4763-a283-2998bed3f4c8,1,159,,My Location,1465107947,0dec1da9-0ad4-4763-a283-2998bed3f4c8-1465107947
27838,2019-01-13,16845f2bd3522f-05283821df56fd8-797f3265-3d10d-...,1,46,Norvan Falls,"49.224433559038886,-122.96509243559368",1547333114,16845f2bd3522f-05283821df56fd8-797f3265-3d10d-...
27637,2018-12-24,167e15b477223d-01d6aa8679517f-293e1d4e-1aeaa0-...,1,146,Quarry Rock,"550 West Hastings Street, Vancouver, BC, Canada",1545645339,167e15b477223d-01d6aa8679517f-293e1d4e-1aeaa0-...
19536,2018-03-01,15fd0f24fa71d-09308a361dbf18-7633301b-2c600-15...,1,55,Big Cedar and Kennedy Falls,"4942 Ruby Street, Vancouver, BC, Canada",1519890029,15fd0f24fa71d-09308a361dbf18-7633301b-2c600-15...
13068,2017-07-03,15d074b7091331-0b95ab55a097ab-8383667-e1000-15...,1,136,Lighthouse Park,"Vancouver, BC V6G 1E5, Canada",1499040860,15d074b7091331-0b95ab55a097ab-8383667-e1000-15...
18050,2017-11-15,15fbe2653271dc-00b881f2a29d3c-7b113d-e1000-15f...,1,55,Big Cedar and Kennedy Falls,"3569 West King Edward Avenue, Vancouver, Brits...",1510695079,15fbe2653271dc-00b881f2a29d3c-7b113d-e1000-15f...
6538,2016-10-09,157a5b05045116-0510a7af9c49508-39126d59-3d10d-...,1,9,,"2170 Bel-Red Road, Bellevue, WA, United States",1476013820,157a5b05045116-0510a7af9c49508-39126d59-3d10d-...
14799,2017-08-02,15da1e47ccf60-00344813f94c22-543d1867-38400-15...,1,146,Quarry Rock,"7786 Ontario Street, Vancouver, BC, Canada",1501670727,15da1e47ccf60-00344813f94c22-543d1867-38400-15...
15650,2017-08-20,15dfd57226974-0c50130835c107-5f2f075c-38400-15...,1,146,Quarry Rock,"2268 West 37th Avenue, Vancouver, BC, Canada",1503275361,15dfd57226974-0c50130835c107-5f2f075c-38400-15...


## Read in the destination data to allow the possibility to pull the correct names

 


In [143]:
f = open("destinations_mapping_Jul-30-18.csv")

reader = csv.reader(f)


destinations = {}


# The index at the end of the for loop just skips the first row which is the header in the csv file
next(reader)
for row in reader:
    destinations[row[0]] = {'name':row[1]}

print(destinations)

{'168': {'name': 'Mammoth Pass - Crater Meadow Trail'}, '178': {'name': 'Sport Climbing at Horseshoe Slabs'}, '181': {'name': 'Hazel Wolf Wetlands Loop'}, '183': {'name': 'Redwood Grove Loop Trail'}, '184': {'name': 'Birdwatching at Fowlsheugh Reserve'}, '12': {'name': 'Point Defiance Loop (OLD)'}, '112': {'name': 'Baden Powell Lynn Canyon to Grouse'}, '175': {'name': 'Rainbow Falls Trail'}, '174': {'name': 'Walking the Town Loop to Sherwins Vista'}, '20': {'name': 'The Lions Binkert Trail'}, '27': {'name': 'Tumamoc Hill'}, '8': {'name': 'Proximity Alert Test'}, '169': {'name': 'McLeod Lake Spur'}, '165': {'name': 'Siskiyou Mountain (White Rabbit Trail)'}, '22': {'name': 'Lake Blanca'}, '11': {'name': 'Carkeek Park'}, '162': {'name': 'Rouge Park Traverse'}, '172': {'name': 'Walking the Lakes Basin Path at Twin Lakes'}, '16': {'name': 'Golden Gardens to Carkeek Park Beach Walk'}, '111': {'name': 'Baden Powell Deep Cove to Lynn Canyon'}, '278': {'name': 'Tujunga Wash Path'}, '13': {'name

# Loop through the rows in the dataframe, geocode, add entry to database

In [144]:
# Loop through the subsetted pandas data frame

# Uncomment the code below to loop through the the sample data frame
# for index, row in sampleDf.iterrows():

for index, row in sampleDf.iterrows():
  

    # Pull out the primary key into a variable
    testKey = row["primary_key"]
    
    # Check to see if that distinctID is in the data base
    # See this post: https://stackoverflow.com/questions/6587879/how-to-elegantly-check-the-existence-of-an-object-instance-variable-and-simultan?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa
    entryExists = session.query(exists().where(Itenerary.distinctKey==testKey)).scalar()

    # If the entry is not in the database
    if not entryExists:
    
        # Get the string to be geocoded
        locationToGeocode = row["startFromLocation"]

        # Try to run the geocode function that returns a dictionary of information
        try:
            geocodeInfo = geocode(locationToGeocode)
            # If geocoding works, set valid to tre
            valid = True

        # If the geocode function doesn't work set valid to false
        except:
            valid = False 

        # If valid is true create a database entry with information from the dataframe, and the returned geocode informaiton
        if valid:
            
            # Sometimes, such as when a generic city is sent to the geocode() function a geometric center
            # is returned, this means there is no postal code
            
            # In this case, we can set the postalCode to One
            if not 'postalCode' in geocodeInfo:
                geocodeInfo['postalCode'] = "none"

            
            # Mapp the gps coordinates returned to the zip code polygons
            zipCodeMapped = mapToPoly(geocodeInfo['lat'], geocodeInfo['lng'])
            
            # TODO:
            # EXPAND THIS MAPPING TO OTHER AREAS
            
            
            ## Get selected Destination Names
            # Pull the selected destination name
            selectedDestinationName = row["selectedDestination_name"]
            
            if not selectedDestinationName:
                
                key = str(row["selectedDestination_id"])
                
                if key in destinations:
            
                    # Pull the data out from the dictionary that was created in the cell above
                    newName = destinations[str(row["selectedDestination_id"])]['name']
        
                    # Add the new name to the new row
                    selectedDestinationName  = newName
            
                # In the case that there is destination that corresponds mark
                else:
                
                    # TODO: CONSIDER CHANGING THE NAME OF THIS TO SOMETHING ELSE
                    selectedDestinationName = "DELETED"
                
            databaseEntry = Itenerary(distinctKey=row["primary_key"],
                                      numberItinerariesReturned=row["numItinerariesReturned"],
                                      selectedDestination_id=row["selectedDestination_id"],
                                      selectedDestination_name=selectedDestinationName,
                                      startFromLocation=row["startFromLocation"],
                                      departureDate=row["departureDate"],
                                      # Get data from python dictionary returned from geocode() function
                                      formatted_address=geocodeInfo['formatted_address'],
                                      lat=geocodeInfo['lat'],
                                      lng=geocodeInfo['lng'],
                                      postalCode=geocodeInfo['postalCode'],
                                      postalCodeMapped=zipCodeMapped,
                                      valid=valid)
        # If valid is false, just fill in the information that we have from the pandas data frame
        else:
            databaseEntry = Itenerary(distinctKey=row["primary_key"],
                                      numberItinerariesReturned=row["numItinerariesReturned"],
                                      selectedDestination_id=row["selectedDestination_id"],
                                      selectedDestination_name=row["selectedDestination_name"],
                                      startFromLocation=row["startFromLocation"],
                                      departureDate=row["departureDate"],
                                      valid=valid)

        # Add the the information to a database.    
        session.add(databaseEntry)
        session.commit()
    
    else:
        print("Entry already inside database")

98164
98007
98101
98102
98101
98201
98004
98101
98045
98164
98164
98102
98029
98119
98118
98115
98164
98102
98201
98133
98122
98121
98119
98164
98164
98004
98102
98101
98007
98122
98121
98004
98109
98102
98164
98164
98103
98121
98121
98164
98105
98201
98101
98188
98103
98059
98164
98164
98105
98029
98115
98121
98164
98164
98164
98164
98164
98112
